In [ ]:

import numpy as np
import sympy as sym
import math as mt

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
def GetLaguerreRecursive(n,x):

    if n==0:
        poly = sym.Pow(1,1)
    elif n==1:
        poly = 1- x
    else:
        poly = ((2*n-1-x)*GetLaguerreRecursive(n-1,x)-(n-1)*GetLaguerreRecursive(n-2,x))/n
   
    return sym.simplify(poly)

def GetDLaguerre(n,x):
    Pn = GetLaguerreRecursive(n,x)
    return sym.diff(Pn,x,1)

def GetNewton(f,df,xn,itmax=10000,precision= 1e-6):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn


def GetRootsGLag(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)
    
        croot = np.round( root, tolerancia )
        
        if croot not in Roots:
            Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

def GetAllRootsGLag(n):

    xn = np.linspace(0, n+(n-1)*np.sqrt(n),100)
    
    Laguerre = []
    DLaguerre = []
    
    for i in range(n+1):
        Laguerre.append(GetLaguerreRecursive(i,x))
        DLaguerre.append(GetDLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLaguerre[n],'numpy')
    Roots = GetRootsGLag(poly,Dpoly,xn)
    
    return Roots


def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)
    
    Laguerre = []
    
    for i in range(n+2):
        Laguerre.append(GetLaguerreRecursive(i,x))
        
    poly = sym.lambdify([x],Laguerre[n+1],'numpy')
    
    Weights = Roots/(((n+1)**2)*(poly(Roots)**2))
    
    return Weights

I = 0
pesos = GetWeightsGLag(n)
raices = GetAllRootsGLag(n)
R = 8.314  
T = np.random.uniform(0, 2000, 10)  
M = 0.002  
v = np.linspace(0, 2000, 1000)

funcion = v * (4 * np.pi * (M / (2 * np.pi * R * T)) ** (3 / 2) * v ** 2 * np.exp(-M * v ** 2 / (2 * R * T)))


for i in range(5):
    
    I += pesos[i]*funcion(raices[i])